In [1]:

STD_ANGLE  = 90
STD_BOUND = 20

# Define hyperparameters
monte_carlo_temp = 10.0  # Increase if acceptance rate is too low
num_relax_rounds = 10  # Increase for more thorough, but slower, optimization
max_relax_iter = 200  # Increase for more thorough, but slower, optimization
num_iterations = 10  # Increase for longer, but slower, runs

#file paths 
temp_file_path = "temp_file.pdb"
render_path = './renders/'


In [2]:
import os
import shutil

def remove_folder_contents(folder_path):
   
    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            if os.path.isfile(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
    else:
        print(f"The folder '{folder_path}' does not exist.")

remove_folder_contents(render_path)

In [3]:
from Bio.PDB import PDBParser, PPBuilder
from Bio.PDB import *
import numpy as np
import random
from tqdm import tqdm

from pyrosetta import *
from pyrosetta.rosetta.core.scoring.constraints import AtomPairConstraint , DihedralConstraint

from pyrosetta.rosetta.core.kinematics import MoveMap
# from pyrosetta.rosetta.protocols.minimization_packing import MinMover

from pyrosetta.rosetta.core.scoring import ScoreType
from pyrosetta.rosetta.core.scoring.func import CircularHarmonicFunc ,HarmonicFunc

from pyrosetta.rosetta.core.id import AtomID

from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.protocols.moves import MonteCarlo


from pyrosetta.rosetta.protocols.moves import RepeatMover



# Load structure
parser = PDBParser()
structure = parser.get_structure('Your_protein', 'test.pdb')

# Get chain A
chain_A = structure[0]['A']



# Save chain A as a separate file
io = PDBIO()
io.set_structure(chain_A)
io.save('base_line_chain_A.pdb')



# Extract sequence
ppb = PPBuilder()
sequence = ""
for pp in ppb.build_peptides(chain_A):
    sequence += str(pp.get_sequence())

print("Orginal Chain Sequence" , sequence)


# Filter out only alpha carbon atoms
alpha_carbons = [atom for residue in chain_A for atom in residue if atom.get_id() == "CA"]

# Define function to calculate distance
def calculate_distance(atom1, atom2):
    diff_vector  = atom2.coord - atom1.coord
    return np.sqrt(np.sum(diff_vector * diff_vector))

# Create distance matrix
num_atoms = len(alpha_carbons)
distance_matrix = np.zeros((num_atoms, num_atoms))

for i in range(num_atoms):
    for j in range(num_atoms):
        distance_matrix[i, j] = calculate_distance(alpha_carbons[i], alpha_carbons[j])

# Save distance matrix to a csv file
np.savetxt("distance_matrix.csv", distance_matrix, delimiter=",")


print("Shape Distance Matrix :" , distance_matrix.shape )



ppb = PPBuilder()

# Using the method you mentioned to get the list of phi and psi angles
phi_psi_list = ppb.build_peptides(chain_A)[0].get_phi_psi_list()

# Separate the phi and psi angles into two lists
phi_angles = [angles[0] for angles in phi_psi_list if angles[0] is not None]
psi_angles = [angles[1] for angles in phi_psi_list if angles[1] is not None]
import math

phi_angles_deg = [math.degrees(angle) for angle in phi_angles]
psi_angles_deg = [math.degrees(angle) for angle in psi_angles]

phi_stdevs = [random.uniform(0, STD_ANGLE) for _ in range(len(phi_angles_deg))]
psi_stdevs = [random.uniform(0, STD_ANGLE) for _ in range(len(psi_angles_deg))]

print(len(phi_angles_deg) , len(psi_angles_deg) , max(phi_angles_deg))


# initialize PyRosetta
init()

# load your protein
pose = pose_from_sequence(sequence)

# Assume phi_angles and psi_angles are your precalculated lists of angles, which are of length N-1

# Set the phi angles for residues 2 to N

for i in range(2, pose.total_residue() + 1):
    pose.set_phi(i, phi_angles_deg[i-2])

# Set the psi angles for residues 1 to N-1
for i in range(1, pose.total_residue()):
    pose.set_psi(i, psi_angles_deg[i-1])

for i in range(len(distance_matrix)):
    for j in range(i+1, len(distance_matrix[i])):
        # create an AtomPairConstraint for the alpha carbons of residues i+1 and j+1
        atom1 = AtomID(2, i+1)  # 2 is the index for alpha carbon ('CA') in Rosetta atom numbering
        atom2 = AtomID(2, j+1)
        func = HarmonicFunc(distance_matrix[i][j], STD_BOUND)  # Harmonic potential with standard deviation of 1.0
        constraint = AtomPairConstraint(atom1, atom2, func)

        # add the constraint to the pose
        pose.add_constraint(constraint)
  
      
for i in range(1, pose.total_residue()):  # Rosetta uses 1-indexing
    # Create AtomID for the atoms forming the psi angle: N(i)-CA(i)-C(i)-N(i+1)
    atom1 = AtomID(1, i)   # N atom of current residue
    atom2 = AtomID(2, i)   # CA atom of current residue
    atom3 = AtomID(4, i)   # C atom of current residue
    atom4 = AtomID(1, i+1) # N atom of next residue if it exists
    
    # Create a CircularHarmonicFunc for the psi angle
    func = CircularHarmonicFunc(psi_angles_deg[i-1], psi_stdevs[i-1])

    # Create a DihedralConstraint for the psi angle
    constraint = DihedralConstraint(atom1, atom2, atom3, atom4, func)

    # Add the constraints to the pose
    pose.add_constraint(constraint)

for i in range(2, pose.total_residue() + 1):  # Rosetta uses 1-indexing
    # Create AtomID for the atoms forming the phi angle: C(i-1)-N(i)-CA(i)-C(i)
    atom1 = AtomID(4, i-1)  # C atom of previous residue
    atom2 = AtomID(1, i)  # N atom of current residue
    atom3 = AtomID(2, i)  # CA atom of current residue
    atom4 = AtomID(4, i)  # C atom of current residue
    
    # Create a CircularHarmonicFunc for the phi angle
    func = CircularHarmonicFunc(phi_angles_deg[i-2], phi_stdevs[i-2])

    # Create a DihedralConstraint for the phi angle
    constraint = DihedralConstraint(atom1, atom2, atom3, atom4, func )

    # Add the constraints to the pose
    pose.add_constraint(constraint)
    


# exit()
# Create a MoveMap that will allow backbone torsion angles to change
movemap = MoveMap()
movemap.set_bb(True)  # True means all backbone torsion angles are allowed to change
movemap.set_chi(True)
# Create a score function
scorefxn = get_score_function()  # Use the default score function
from pyrosetta.rosetta.core.scoring import fa_atr, fa_rep, fa_sol, fa_intra_rep, fa_elec, pro_close, hbond_sr_bb, hbond_lr_bb, hbond_bb_sc, hbond_sc, constraints

# Increase the weights of the constraint terms
scorefxn.set_weight(ScoreType.atom_pair_constraint, 10.0)
scorefxn.set_weight(ScoreType.dihedral_constraint, 10.0)

#consider other functions

scorefxn.set_weight(fa_atr, 0.8)
scorefxn.set_weight(fa_rep, 0.44)
scorefxn.set_weight(fa_sol, 0.75)
scorefxn.set_weight(fa_intra_rep, 0.004)
scorefxn.set_weight(fa_elec, 0.70)
scorefxn.set_weight(pro_close, 1.25)
scorefxn.set_weight(hbond_sr_bb, 0.5)
scorefxn.set_weight(hbond_lr_bb, 0.5)
scorefxn.set_weight(hbond_bb_sc, 0.5)
scorefxn.set_weight(hbond_sc, 1.0)



# Create a Monte Carlo mover
mc = MonteCarlo(pose, scorefxn, monte_carlo_temp)  # The last parameter is the temperature

# Create a FastRelax mover
fast_relax = FastRelax(scorefxn, num_relax_rounds)  # The second parameter is the number of rounds
fast_relax.set_movemap(movemap)
fast_relax.max_iter(max_relax_iter)
# fast_relax.max_iter(100000000)  # Set maximum iterations
fast_relax.set_scorefxn(scorefxn)




Orginal Chain Sequence PQITLWQRPLVTIKIGGQLKEALLNTGADDTVLEEMSLPGRWKPKMIGGIGGFIKVRQYDQILIEICGHKAIGTVLVGPTPVNIIGRNLLTQIGCTLNF
Shape Distance Matrix : (99, 99)
98 98 171.86695329100388
PyRosetta-4 2023 [Rosetta PyRosetta4.Release.python310.m1 2023.27+release.e3ce6ea9faf661ae8fa769511e2a9b8596417e58 2023-07-07T12:00:46] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.python310.m1 r353 2023.27+release.e3ce6ea9faf e3ce6ea9faf661ae8fa769511e2a9b8596417e58 http://www.pyrosetta.org 2023-07-07T12:00:46
core.init: command: PyRosetta -ex1 -ex2aro -database /Users/ahora/anaconda3/lib/python3.10/site-packages/pyrosetta-2023.27+release.e3ce6ea9faf-py3.10-macosx-11.0-arm64.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom', seed=-370177585 seed_o

/Users/ahora/anaconda3/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain A is discontinuous at line 1970.
  warnings.warn(
/Users/ahora/anaconda3/lib/python3.10/site-packages/Bio/PDB/StructureBuilder.py:89: PDBConstructionWarning: WARNING: Chain B is discontinuous at line 2038.
  warnings.warn(


core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 0.541447 seconds.
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_param

In [4]:
#Save File Without Any Change
pose.dump_pdb(os.path.join( render_path , f'before_anything.pdb'))

True

In [5]:
from tqdm import tqdm

for i in tqdm(range(num_iterations)):  
    fast_relax.apply(pose)
    mc.boltzmann(pose)

    # Convert the pose to a PDB string
    pose.dump_pdb(os.path.join( render_path , f'render_{i}.pdb'))
    

  0%|          | 0/10 [00:00<?, ?it/s]

protocols.relax.FastRelax: CMD: repeat  90997.7  0  0  0.44
protocols.relax.FastRelax: CMD: coord_cst_weight  90997.7  0  0  0.44
protocols.relax.FastRelax: CMD: scale:fa_rep  18726  0  0  0.0176
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1902 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  13363.2  0  0  0.0176
protocols.relax.FastRelax: CMD: scale:fa_rep  13746.2  0  0  0.02244
protocols.relax.FastRelax: CMD: min  350.745  7.50603  7.50603  0.02244
protocols.relax.FastRelax: CMD: coord_cst_weight  350.745  7.50603  7.50603  0.02244
protocols.relax.FastRelax: CMD: scale:fa_rep  1858.48  7.50603  7.50603  0.1166
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1364 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protoc

 10%|█         | 1/10 [01:21<12:09, 81.07s/it]

protocols.relax.FastRelax: CMD: repeat  11.9936  7.54146  0  0.44
protocols.relax.FastRelax: CMD: coord_cst_weight  11.9936  7.54146  0  0.44
protocols.relax.FastRelax: CMD: scale:fa_rep  -45.9796  7.54146  0  0.0176
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1638 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -51.6183  7.54146  0  0.0176
protocols.relax.FastRelax: CMD: scale:fa_rep  -50.4256  7.54146  0  0.02244
protocols.relax.FastRelax: CMD: min  -88.4696  7.43906  0.384968  0.02244
protocols.relax.FastRelax: CMD: coord_cst_weight  -88.4696  7.43906  0.384968  0.02244
protocols.relax.FastRelax: CMD: scale:fa_rep  -0.75205  7.43906  0.384968  0.1166
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1530 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Ins

 20%|██        | 2/10 [02:36<10:22, 77.81s/it]

...

 80%|████████  | 8/10 [10:17<02:32, 76.36s/it]

protocols.relax.FastRelax: CMD: repeat  11.9556  7.54143  0  0.44
protocols.relax.FastRelax: CMD: coord_cst_weight  11.9556  7.54143  0  0.44
protocols.relax.FastRelax: CMD: scale:fa_rep  -45.9874  7.54143  0  0.0176
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1569 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -51.5792  7.54143  0  0.0176
protocols.relax.FastRelax: CMD: scale:fa_rep  -50.4007  7.54143  0  0.02244
protocols.relax.FastRelax: CMD: min  -91.948  7.44794  0.4297  0.02244
protocols.relax.FastRelax: CMD: coord_cst_weight  -91.948  7.44794  0.4297  0.02244
protocols.relax.FastRelax: CMD: scale:fa_rep  1.74695  7.44794  0.4297  0.1166
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1676 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiatin

 90%|█████████ | 9/10 [11:32<01:16, 76.10s/it]

protocols.relax.FastRelax: CMD: repeat  11.9884  7.54133  0  0.44
protocols.relax.FastRelax: CMD: coord_cst_weight  11.9884  7.54133  0  0.44
protocols.relax.FastRelax: CMD: scale:fa_rep  -45.9923  7.54133  0  0.0176
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1638 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
protocols.relax.FastRelax: CMD: repack  -51.6312  7.54133  0  0.0176
protocols.relax.FastRelax: CMD: scale:fa_rep  -50.4384  7.54133  0  0.02244
protocols.relax.FastRelax: CMD: min  -92.1893  7.44893  0.428944  0.02244
protocols.relax.FastRelax: CMD: coord_cst_weight  -92.1893  7.44893  0.428944  0.02244
protocols.relax.FastRelax: CMD: scale:fa_rep  0.241929  7.44893  0.428944  0.1166
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 1686 rotamers at 99 positions.
core.pack.interaction_graph.interaction_graph_factory: Ins

100%|██████████| 10/10 [12:47<00:00, 76.79s/it]


In [6]:
pose.dump_pdb("output_chain_a_final.pdb")

True